# Yelp Dataset Analysis: Philadelphia Restaurants

## Data Loading and Preprocessing

I use three parts of the Yelp Open Dataset: the business file `yelp_academic_dataset_business.json`, the check-in file `yelp_academic_dataset_checkin.json`, and the tip file `yelp_academic_dataset_tip.json`. The business file provides each restaurant’s basic information, ratings, review counts, and open status. The check-in file contains timestamped user check-ins, which I convert into a simple count. The tip file contains short user tips, which I summarize into a tip count and an average compliment count.


I merge these three datasets, keep only restaurants, fill missing engagement values with zeros, and standardize ZIP codes and city names. I then merge in ZIP-level income information from `income-zip.csv` and filter the data to restaurants located in Philadelphia. After removing rows without coordinates, I aggregate to the ZIP level to compute restaurant counts, the share still open, and average engagement metrics. I then merge this summary with the ZIP boundary GeoJSON file `zipcodes_poly.geojson` to prepare the data for mapping and save all cleaned tables for later analysis.

In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
from pathlib import Path

DATA_DIR = Path("../data")

# Load Yelp business file and keep restaurants
# We keep the following columns from `yelp_academic_dataset_business.json`: 
# `business_id`, `name`, `address`, `city`, `state`, `postal_code`, `latitude`, 
# `longitude`, `stars`, `review_count`, `is_open`, `attributes`, `categories`, `hours`.
biz_path = DATA_DIR / "yelp_academic_dataset_business.json"
biz = pd.read_json(biz_path, lines=True)
biz = biz[biz["categories"].str.contains("Restaurant", na=False)].copy()


# Load Yelp check-in file and summarize
checkin_path = DATA_DIR / "yelp_academic_dataset_checkin.json"
checkin = pd.read_json(checkin_path, lines=True)

# Each row has business_id and a comma-separated list of timestamps in "date"
checkin["checkin_count"] = checkin["date"].apply(
    lambda x: len(x.split(", ")) if isinstance(x, str) else 0
)
checkin = checkin[["business_id", "checkin_count"]]

# Load Yelp tip file and summarize per business
tip_path = DATA_DIR / "yelp_academic_dataset_tip.json"
tip = pd.read_json(tip_path, lines=True)

tip_summary = (
    tip.groupby("business_id")
       .agg(
           tip_count=("text", "count"),
           avg_compliments=("compliment_count", "mean")
       )
       .reset_index()
)


# Merge Yelp business + checkin + tip
yelp = (
    biz.merge(checkin, on="business_id", how="left")
       .merge(tip_summary, on="business_id", how="left")
       .copy()
)

yelp["checkin_count"]   = yelp["checkin_count"].fillna(0).astype(int)
yelp["tip_count"]       = yelp["tip_count"].fillna(0).astype(int)
yelp["avg_compliments"] = yelp["avg_compliments"].fillna(0.0)
yelp["city"] = yelp["city"].astype(str).str.lower()
yelp["postal_code"] = yelp["postal_code"].astype(str).str[:5].str.zfill(5)

# Load Income by ZIP and keep many columns for future use
inc_path = DATA_DIR / "income-zip.csv"  # your CSV from RowZero
income = pd.read_csv(inc_path)


rename_map = {
    "ZIP": "zip",
    "Total population": "population",
    "Households - Median income (dollars)": "median_income",
    "Households - Mean income (dollars)": "mean_income",
}
income = income.rename(columns=rename_map)


income["zip"] = income["zip"].astype(str).str.zfill(5)

# Merge income onto Yelp by ZIP
merged = yelp.merge(income, left_on="postal_code", right_on="zip", how="left")


# Filter to Philadelphia city proper
philly = merged[merged["city"] == "philadelphia"].copy()


philly = philly.dropna(subset=["latitude", "longitude"])
fill_zeros = ["checkin_count", "tip_count", "avg_compliments"]
philly[fill_zeros] = philly[fill_zeros].fillna(0)

if philly["median_income"].isna().any():
    philly["median_income"] = philly["median_income"].fillna(philly["median_income"].median())

# Aggregate to ZIP for choropleths and city summaries
zip_summary = (
    philly.groupby("postal_code")
          .agg(
              n_businesses = ("business_id", "nunique"),
              pct_open     = ("is_open", "mean"),
              avg_stars    = ("stars", "mean"),
              avg_reviews  = ("review_count", "mean"),
              avg_checkins = ("checkin_count", "mean"),
              avg_tips     = ("tip_count", "mean"),
              avg_comps    = ("avg_compliments", "mean"),
              median_income= ("median_income", "mean")
          )
          .reset_index()
)

# Join ZIP polygons for mapping and save reusable files
geo_path = DATA_DIR / "zipcodes_poly.geojson"  # OpenDataPhilly "ZIP Codes – Polygon (GeoJSON)"
geo = gpd.read_file(geo_path)
geo["CODE"] = geo["CODE"].astype(str).str.zfill(5)

merged_geo = geo.merge(zip_summary, left_on="CODE", right_on="postal_code", how="left")

# Save quick artifacts so you can reload without recomputing
DATA_DIR.mkdir(parents=True, exist_ok=True)
philly_out      = DATA_DIR / "philly_yelp_income_clean.csv"
zip_summary_out = DATA_DIR / "philly_zip_summary.csv"
geo_out         = DATA_DIR / "merged_philly.geojson"

philly.to_csv(philly_out, index=False)
zip_summary.to_csv(zip_summary_out, index=False)
merged_geo.to_file(geo_out, driver="GeoJSON")

print("Rows in Yelp restaurants:", len(yelp))
print("Rows in Philadelphia subset:", len(philly))
print("ZIPs in summary:", len(zip_summary))
print("Geo polygons merged:", len(merged_geo))
print("Saved:", philly_out.name, zip_summary_out.name, geo_out.name)


Rows in Yelp restaurants: 52286
Rows in Philadelphia subset: 5856
ZIPs in summary: 57
Geo polygons merged: 48
Saved: philly_yelp_income_clean.csv philly_zip_summary.csv merged_philly.geojson


### **Mapping Restaurant Survival by ZIP**

Here I load the merged_philly.geojson file and plot a map of Philadelphia ZIP codes. Each area is colored by the share of restaurants that are still open. The tooltip shows the ZIP code, how many restaurants are in that area, and the percent that remain open. This gives a quick spatial view of which parts of the city have higher or lower survival rates.

In [6]:
import altair as alt

geo = alt.Data(
    url='../data/merged_philly.geojson',
    format={'type': 'json', 'property': 'features'}  # <-- important
)

pct_open_map = (
    alt.Chart(geo)
      .mark_geoshape(stroke='white', strokeWidth=0.5)
      .encode(
          color=alt.Color('properties.pct_open:Q', title='Share open',
                          scale=alt.Scale(scheme='greens')),
          tooltip=[
              alt.Tooltip('properties.postal_code:N', title='ZIP'),
              alt.Tooltip('properties.n_businesses:Q', title='# restaurants', format=','),
              alt.Tooltip('properties.pct_open:Q', title='Share open', format='.2f')
          ]
      )
      .properties(title='Philadelphia: Share of Restaurants Still Open by ZIP',
                  width=650, height=450)
      .project('mercator')
)
pct_open_map


alt.Chart(...)

### Mapping Median Income by ZIP

I also map median household income using the same ZIP code shapes from `merged_philly.geojson`. Each ZIP is colored by its median income to show how wealth varies across Philadelphia. The tooltip lists the ZIP code and its income value. Having this map next to the restaurant survival map makes it easy to compare economic patterns with where restaurants tend to stay open.

In [7]:
income_map = (
    alt.Chart(geo)
      .mark_geoshape(stroke='white', strokeWidth=0.5)
      .encode(
          color=alt.Color('properties.median_income:Q', title='Median income ($)',
                          scale=alt.Scale(scheme='blues')),
          tooltip=[
              alt.Tooltip('properties.postal_code:N', title='ZIP'),
              alt.Tooltip('properties.median_income:Q', title='Median income', format=',')
          ]
      )
      .properties(title='Philadelphia: Median Household Income by ZIP',
                  width=650, height=450)
      .project('mercator')
)
income_map


alt.Chart(...)

In [5]:
import geopandas as gpd

geo = gpd.read_file("../data/merged_philly.geojson")
print(geo.columns.tolist())
print(geo.head(3))


['OBJECTID', 'CODE', 'COD', 'Shape__Area', 'Shape__Length', 'postal_code', 'n_businesses', 'pct_open', 'avg_stars', 'avg_reviews', 'avg_checkins', 'avg_tips', 'avg_comps', 'median_income', 'geometry']
   OBJECTID   CODE  COD   Shape__Area  Shape__Length postal_code  \
0         1  19120   20  1.456207e+07   19887.714114       19120   
1         2  19121   21  1.102598e+07   15728.621590       19121   
2         3  19122   22  5.689181e+06    9599.539345       19122   

   n_businesses  pct_open  avg_stars  avg_reviews  avg_checkins  avg_tips  \
0          88.0  0.795455   3.357955    30.170455     50.761364  5.204545   
1          61.0  0.557377   3.204918    35.557377     47.540984  4.786885   
2         103.0  0.669903   3.815534    63.485437     87.009709  7.553398   

   avg_comps  median_income                                           geometry  
0   0.010085        51993.0  POLYGON ((-75.11107 40.04682, -75.10943 40.045...  
1   0.005941        36208.0  POLYGON ((-75.19227 39.994

In [12]:
# use the ZIP-level table you already built
g = zip_summary.copy()

scatter = (
    alt.Chart(g)
      .mark_circle()
      .encode(
          x=alt.X('median_income:Q', title='Median Household Income ($)'),
          y=alt.Y('pct_open:Q', title='Share of Restaurants Still Open'),
          size=alt.Size('n_businesses:Q', title='# of Restaurants'),
          tooltip=['postal_code:N','median_income:Q','pct_open:Q','n_businesses:Q']
      )
      .properties(title='Income vs. Restaurant Survival in Philadelphia',
                  width=500, height=350)
)
scatter


alt.Chart(...)